# __User Tasks__
The frontend will also handle a lot of validation and error handling, so pretend this is the best-case scenario. We can walk through the logical order of things on the frontend. 
The frontend will also have to handle multi-user activity, but only in so far as what user_id variable is currently active, etc. Simple things. As long as Firebase functions can handle simultaneous activity, we're good. 

In [3]:
# Set kernel to "critics (Python 3.12.4)"

import os
import sys
from dotenv import load_dotenv
from firebase_admin import credentials
import firebase_admin

# Add project root and utils to path
sys.path.append('/Users/geneva/Downloads/PERSONAL/critics')
# sys.path.append('/Users/geneva/Downloads/PERSONAL/critics/utils')

from utils.user import *
from utils.board import *
from utils.firebase import *
from utils.restaurant import *
from utils.category import *

# Load environment variables
load_dotenv()

def initialize_firebase():
    """Initialize Firebase connection using credentials from environment"""
    service_account_path = os.getenv('SERVICE_ACCOUNT_PATH')
    database_url = os.getenv('DATABASE_URL')

    if not service_account_path:
        raise ValueError("SERVICE_ACCOUNT_PATH environment variable is not set.")

    if not firebase_admin._apps:
        cred = credentials.Certificate(service_account_path)
        firebase_admin.initialize_app(cred, {
            'databaseURL': database_url
        })

# Initialize
initialize_firebase()

# __Board and User Structure__

In [4]:
# New user is created when they first open the app. 

name = "Geneva"
user_id = "user_101"  

new_user = create_user(name, user_id)
print(f"Created new user: {new_user}")

Created new user: {'name': 'Geneva', 'boards': [], 'last_active': None}


In [3]:
# User creates a new board, and names it "This is a test board"

user_id = "user_001"
board_id = "board_001"  
board_name = "Genny's Board"

# create the board 
new_board = create_board(board_id, board_name, user_id) # user_id from the prev. block

#1. Add the board to the user's boards
updated_boards = add_board_to_user(user_id, board_id)

#2. Add the user as a member of the board
add_user_to_board(board_id, user_id)

print(f"Created new board: {new_board}")


ValueError: User user_001 is already a member of board board_001.

In [9]:
# New user is added, then joins the board. 

name = "Chris"
user_id = "user_002"  

new_user = create_user(name, user_id)
print(f"Created new user: {new_user}")

#1. Add this board to chris's boards
updated_boards = add_board_to_user(user_id, board_id)  # board_id from previous cell

#2. Add chris to the list of board's members
add_user_to_board(board_id, user_id)

print(f"Added {name} to board: {board_id}")


Created new user: {'name': 'Chris', 'boards': [], 'last_active': None}
Added Chris to board: board_001


In [6]:
# Geneva adds a category to the board.

board_id = "board_001" 
category_id = "category_001"
category_name = "Fine Dining"
caption = "Can't tell if we go to dine, or to dress up."

# create the category
new_category = create_category(category_id, category_name, caption)

# add the category to the board's list of categories
link_category_to_board(board_id, category_id)

In [3]:
# Geneva tries to delete Chris's board

user_id = "user_001"
board_id = "board_001"

delete_board(board_id, user_id)


In [5]:
# Chris leaves Genny's board

user_id = "user_002"
board_id = "board_001"

remove_user_from_board(board_id, user_id)

# __V1 Restaurant Structure__

In [2]:
# Geneva adds a restaurant to the board. 

category_id = "category_001" # Fine Dining 
restaurant_id = "restaurant_001" 

data = {
    "name": "The Modern",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "The food was good, but the service was a bit slow.",
    "visits": ["2023-05-15", "2023-07-22", "2023-09-03"],
    "location": "9 W 57th St, New York, NY 10019",
    "dishes": ["Roasted Beet Salad", "Filet Mignon", "Chocolate Cake"],
    "photo": "https://www.themodern.com/images/the-modern-restaurant-exterior.jpg"
}

new_restaurant = add_restaurant_to_category(category_id, restaurant_id, data)

In [4]:
# Chris adds a restaurant to the board. 

category_id = "category_001" # Fine Dining 
restaurant_id = "restaurant_002" 

data = {
    "name": "Le Bernardin",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "Exceptional seafood tasting menu. Service was impeccable.",
    "visits": ["2023-08-12"],
    "location": "155 W 51st St, New York, NY 10019",
    "dishes": ["Tuna Tartare", "Lobster", "Chocolate-Olive Oil Dessert"],
    "photo": "https://www.le-bernardin.com/uploads/images/gallery/exterior.jpg"
}

new_restaurant = add_restaurant_to_category(category_id, restaurant_id, data)



In [2]:
# Chris changes a restaurant's rating. 

category_id = "category_001"
restaurant_id = "restaurant_001"
new_rating = 9.1

edit_rating(category_id, restaurant_id, rating_1=new_rating)



{'rating_1': 9.1}

In [5]:
# Geneva adds another category

board_id = "board_001"
category_id = "category_002"
name = "Casual Dining"
caption = "Just another weekend"

new_category = create_category(category_id, name, caption)
link_category_to_board(board_id, category_id)


In [2]:
# Geneva adds a dish from a restaurant. 

category_id = "category_001"
restaurant_id = "restaurant_001"
dish = "Bipimbap"

add_dish(category_id, restaurant_id, dish)



In [3]:
# She removes it 

category_id = "category_001"
restaurant_id = "restaurant_001"
dish = "Bipimbap"

delete_dish(category_id, restaurant_id, dish)


In [4]:
# Geneva switches a restaurant's category. 

old_category_id = "category_001"
new_category_id = "category_002"
restaurant_id = "restaurant_001"

switch_restaurant_category(old_category_id, new_category_id, restaurant_id)

# __V2 Restaurant Structure:__ Independent of category

In [2]:
# testing new add restaurant function

category_id = "category_001"
restaurant_id = "restaurant_003"

data = {
    "name": "The Modern",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "The food was good, but the service was a bit slow.",
    "visits": ["2023-05-15", "2023-07-22", "2023-09-03"],
    "location": "9 W 57th St, New York, NY 10019",
    "dishes": ["Roasted Beet Salad", "Filet Mignon", "Chocolate Cake"],
    "photo": "https://www.themodern.com/images/the-modern-restaurant-exterior.jpg"
}

new_restaurant = add_restaurant_to_category(category_id, restaurant_id, data)

In [3]:
# edit rating 

restaurant_id = "restaurant_003"
new_rating = 9.2

edit_rating(restaurant_id, 1, new_rating)

{'rating_1': 9.2}

In [2]:
# edit notes 

restaurant_id = "restaurant_003"
new_notes = "The food was great, but the service was a bit slow."

edit_notes(restaurant_id, new_notes)

{'notes': 'The food was great, but the service was a bit slow.'}

In [2]:
# add visit 

restaurant_id = "restaurant_003"
visit_date = "2025-01-12"

add_visit(restaurant_id, visit_date)

['2023-05-15', '2023-07-22', '2023-09-03', '2025-01-12']

In [3]:
# delete visit 

restaurant_id = "restaurant_003"

delete_visit(restaurant_id)

['2023-05-15', '2023-07-22', '2023-09-03']

In [4]:
# edit dish ranking 

restaurant_id = "restaurant_003"
new_dishes = ["Filet Mignon", "Roasted Beet Salad", "Chocolate Cake"]

edit_dish_ranking(restaurant_id, new_dishes)

['Filet Mignon', 'Roasted Beet Salad', 'Chocolate Cake']

In [5]:
# add dish 

restaurant_id = "restaurant_003"
dish = "Bipimbap"

add_dish(restaurant_id, dish)


['Filet Mignon', 'Roasted Beet Salad', 'Chocolate Cake', 'Bipimbap']

In [6]:
# delete dish 

restaurant_id = "restaurant_003"
dish = "Bipimbap"

delete_dish(restaurant_id, dish)

['Filet Mignon', 'Roasted Beet Salad', 'Chocolate Cake']

In [4]:
# switch restaurant category 

new_category_id = "category_002"
restaurant_id = "restaurant_003"

switch_restaurant_category(restaurant_id, new_category_id)

{'category_code': 'category_002',
 'dishes': ['Filet Mignon', 'Roasted Beet Salad', 'Chocolate Cake'],
 'location': '9 W 57th St, New York, NY 10019',
 'name': 'The Modern',
 'notes': 'The food was great, but the service was a bit slow.',
 'photo': 'https://www.themodern.com/images/the-modern-restaurant-exterior.jpg',
 'rating_1': 9.2,
 'rating_2': 8,
 'rating_3': 7,
 'visits': ['2023-05-15', '2023-07-22', '2023-09-03']}

# __V3 Updates to Category Structure__

In [2]:
# create new board 

board_id = "board_001"
name = "Genny's Board"
owner = "user_001"

new_board = create_board(board_id, name, owner)


In [12]:
# create new category 

board_id = "board_001"
category_id = "category_003"
name = "Fast Food"
caption = "Fast food is good food"

new_category = create_category(category_id, name, caption)
link_category_to_board(board_id, category_id)

In [8]:
# create new restaurant 

category_id = "category_003"
restaurant_id = "restaurant_004"

data = {
    "name": "The Modern",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "The food was good, but the service was a bit slow.",
    "visits": ["2023-05-15", "2023-07-22", "2023-09-03"],
    "location": "9 W 57th St, New York, NY 10019",
    "dishes": ["Roasted Beet Salad", "Filet Mignon", "Chocolate Cake"],
    "photo": "https://www.themodern.com/images/the-modern-restaurant-exterior.jpg"
}

new_restaurant = create_restaurant(category_id, restaurant_id, data)

In [9]:
# create another restaurant 

category_id = "category_003"
restaurant_id = "restaurant_005"

data = {
    "name": "The Modern",
    "rating_1": 9,
    "rating_2": 8,
    "rating_3": 7,
    "notes": "The food was good, but the service was a bit slow.",
    "visits": ["2023-05-15", "2023-07-22", "2023-09-03"],
    "location": "9 W 57th St, New York, NY 10019",
    "dishes": ["Roasted Beet Salad", "Filet Mignon", "Chocolate Cake"],
    "photo": "https://www.themodern.com/images/the-modern-restaurant-exterior.jpg"
}

new_restaurant = create_restaurant(category_id, restaurant_id, data)

In [6]:
# remove restaurant from category 

category_id = "category_003"
restaurant_id = "restaurant_004"

delete_restaurant(restaurant_id)

In [2]:
# test delete category 

board_id = "board_001"
category_id = "category_003"

delete_category(category_id, board_id)

In [6]:
# delete board 

user_id = "user_001"
board_id = "board_001"

delete_board(board_id, user_id)

In [2]:
# create a new user, properly:

name = "Chris"
user_id = "user_002"

new_user = create_user(name, user_id)


In [4]:
# add chris to board

user_id = "user_002"
board_id = "board_001"

add_user_to_board(board_id, user_id)


In [5]:
# chris deletes himself as a user

user_id = "user_002"
board_id = "board_001"

delete_user(user_id)


In [2]:
# a new user comes in and creates a new board. 

name = "jay"
user_id = "user_003"

new_user = create_user(name, user_id)

# create a new board 

board_id = "board_002"
name = "Jay's Board"
owner = "user_003"

new_board = create_board(board_id, name, owner)

In [3]:
# jay deletes himself as a user

user_id = "user_003"

delete_user(user_id)

# __Testing Category Edit Functions__

In [2]:
# edit category name 

category_id = "category_003"
new_name = "Fine Dining"
caption = "Can't tell if we go to dine, or to dress up."

edit_category_name_caption(category_id, new_name, caption)